In [1]:
from pathlib import Path

DATA_DIR = Path('"C:/Users/olein/Documents/EcoDonut/russia_vectors/fabdem_data"')    # папка с распакованными тайлами FABDEM (*.tif)
OUT_DIR  = Path('fabdem_tiles_russia')                   # куда писать результаты (GeoParquet)
LOG_CSV  = Path('fabdem_processing_log.csv')             # CSV лог
TILES_INDEX = Path('example_data/fabdem_tiles.parquet')          # индекс тайлов (геометрии + file_name)
REGIONS_GEOJSON = Path('example_data/russia_regions.geojson')                     # регионы РФ (полигональная маска)

OUT_DIR.mkdir(parents=True, exist_ok=True)

print('DATA_DIR =', DATA_DIR.resolve())
print('OUT_DIR  =', OUT_DIR.resolve())
print('LOG_CSV  =', LOG_CSV.resolve())


DATA_DIR = C:\Users\olein\Documents\EcoDonut\example\"C:\Users\olein\Documents\EcoDonut\russia_vectors\fabdem_data"
OUT_DIR  = C:\Users\olein\Documents\EcoDonut\example\fabdem_tiles_russia
LOG_CSV  = C:\Users\olein\Documents\EcoDonut\example\fabdem_processing_log.csv


In [2]:

from ecodonut.landscape_vectorizer import vectorize_fabdem_tiles
import geopandas as gpd
import pandas as pd

tiles_gdf = gpd.read_parquet(TILES_INDEX)
regions_gdf = gpd.read_file(REGIONS_GEOJSON)

if tiles_gdf.crs != regions_gdf.crs:
    regions_gdf = regions_gdf.to_crs(tiles_gdf.crs)


In [ ]:


REGION_NAME_COL = 'Наименование'
example_region_name = regions_gdf.iloc[0][REGION_NAME_COL]

filter_gdf = regions_gdf[regions_gdf[REGION_NAME_COL] == example_region_name][['geometry']].copy()

# Параметры векторизации
height_step = 5.0          # шаг по высоте (м) для изолиний/полигонов
slope_step_deg = 5.0       # шаг классов уклона (градусы)
smooth_sigma_slope = 1.0   # сглаживание перед расчётом уклона
aspect_step_deg = 90.0     # ширина бинов аспекта (напр., 90 -> N/E/S/W)
smooth_sigma_aspect = 1.0  # сглаживание перед расчётом аспекта

max_workers = 6            # число процессов-воркеров
skip_existing = True       # не пересчитывать уже существующие тайлы

# Режимы отбора тайлов:
#  1) Все .tif из DATA_DIR:    tiles_gdf=None, filter_gdf=None
#  2) По регионам (пересечение): tiles_gdf=tiles_gdf, filter_gdf=filter_gdf

vectorize_fabdem_tiles(
    data_dir=DATA_DIR,
    out_dir=OUT_DIR,
    log_csv=LOG_CSV,
    tiles_gdf=tiles_gdf,
    filter_gdf=filter_gdf,
    height_step=height_step,
    slope_step_deg=slope_step_deg,
    smooth_sigma_slope=smooth_sigma_slope,
    aspect_step_deg=aspect_step_deg,
    smooth_sigma_aspect=smooth_sigma_aspect,
    max_workers=max_workers,
    skip_existing=skip_existing,
)

print('Готово. Лог:', LOG_CSV.resolve())


In [ ]:

log_df = pd.read_csv(LOG_CSV, dtype=str)
display(log_df.tail(10))

# Быстрый сбор примеров слоёв из OUT_DIR
from pathlib import Path
some_layers = list(Path(OUT_DIR).glob('*_height_polygons_*m.parquet'))[:3]
some_layers


In [ ]:

# Превью одного слоя (если найден)
import geopandas as gpd

if some_layers:
    sample = some_layers[0]
    print('Preview layer:', sample)
    gdf = gpd.read_parquet(sample)
    display(gdf.head())
    # gdf.explore(column='height')  # интерактивная карта (в Jupyter Lab/Notebook с поддержкой)
else:
    print('Нет примеров слоёв для превью.')



### Заметки
- Выходные файлы пишутся в формате **GeoParquet** в папку `OUT_DIR` (один файл на тайл и тип слоя).
- Имена формируются так:
  - `{tile}_height_iso_lines_{height_step}m.parquet`
  - `{tile}_height_polygons_{height_step}m.parquet`
  - `{tile}_slope_deg_polygons_{slope_step_deg}deg.parquet`
  - `{tile}_aspect_{aspect_step_deg}deg_polygons.parquet`
- Инкрементальный лог — CSV с путями и ошибками по каждому тайлу: `LOG_CSV`.
- Повторный запуск с `skip_existing=True` пропускает уже рассчитанные тайлы.
